# Puzzles

## "Easy as ABC"
Source: https://twitter.com/Five_Triangles/status/1015738379454046208

Rules:
Letters A,B,C must each appear exactly once in every row and column.
Outside letters must be closest to that letter in the respective row or column, e.g., C must be highest in column 1.

![The puzzle](https://pbs.twimg.com/media/DhigAqEXkAAsGtN.jpg)

### My solution

In [1]:
from copy import deepcopy

In [2]:
def checkrow(row, req):
    '''Make sure a given row (or column) isn't breaking its two given requirements.'''

    # The row may contain up to 1 A, 1 B, 1 C, and the rest can be spaces
    n = len(row)
    if row.count('A') > 1 or row.count('B') > 1 or row.count('C') > 1 or row.count(' ') > n-3:
        return False
    
    # If the row has any letters, check that the first one matches the requirement
    rowstr = ''.join(row)
    if len(rowstr.strip()) > 0 and req[0] and rowstr.strip()[0] != req[0]:
        return False
    
    # If the row is full, check that the last letter matches the requirement
    if len(rowstr) == len(row) and req[1] and rowstr.strip()[-1] != req[1]:
        return False
    
    return True

def cols(grid):
    '''Get all the columns of a grid (essentially, transpose the grid).'''
    return [[e[i] for e in grid] for i in range(len(grid[0]))]

def col(grid, k):
    '''Get a single column of a grid.'''
    return [e[k] for e in grid]

def checkgrid(grid, row_reqs, col_reqs):
    '''Check that no rules are violated in an entire grid.'''
    if not all(checkrow(g, r) for g,r in zip(grid, row_reqs)):
        return False
    if not all(checkrow(g, r) for g,r in zip(cols(grid), col_reqs)):
        return False
    return True

def printgrid(grid):
    '''Neatly print out a grid.'''
    print(('\n' + '-' * (4*len(grid[0]) - 3) + '\n')
          .join([' | '.join(g) for g in grid]))  

# In each spot in a grid, letters will be tested in this order
cycle = {'':' ', ' ':'A', 'A':'B', 'B':'C'}
    
def solve_it(row_reqs, col_reqs):
    '''For an ABC grid problem specified by certain row and 
    column requirements, find and display all solutions.'''

    solutions = []
    
    # Size of the problem
    n = len(row_reqs)

    # The initial grid (all blank)
    grid = [[''] * n for _ in range(n)]

    # Starting position in the grid (upper left)
    currx, curry = 0, 0

    # Keep working until all possibilites are exhausted
    # (so if there are multiple solutions, we can find all of them)
    while curry >= 0:

        # At the current position in the grid, try the next letter
        try:
            grid[curry][currx] = cycle[grid[curry][currx]]

        # If there are no more letters to try, back up to the previous position
        except KeyError: 
            grid[curry][currx] = ''
            currx -= 1
            if currx < 0:
                currx  = n-1
                curry -= 1
            continue

        # If we have found a solution, save it
        if (currx, curry) == (n-1,n-1) and checkgrid(grid, row_reqs, col_reqs):
            solutions.append(deepcopy(grid))
            
        # Otherwise, if we're not currently breaking any rules, proceed to the next position
        elif (checkrow(grid[curry], row_reqs[curry]) and 
              checkrow(col(grid, currx), col_reqs[currx])):
            currx += 1
            if currx > n-1:
                currx  = 0
                curry += 1
    
    return solutions


In [3]:
soln = solve_it(
    row_reqs = [(None, None),
                ('A', None),
                (None, 'A'),
                (None, 'B'),
                (None, 'B'),
                (None, None)],
    col_reqs = [('C', None),
                ('C', 'A'),
                ('A', None),
                (None, None),
                (None, 'B'),
                (None, 'A')]
)

for s in soln: 
    printgrid(s)
    print('\n\n')


  | C |   |   | A | B
---------------------
  |   | A | B |   | C
---------------------
  | B |   |   | C | A
---------------------
C | A | B |   |   |  
---------------------
A |   |   | C | B |  
---------------------
B |   | C | A |   |  





## "1 to 9" Puzzle #180713
Source: https://twitter.com/1to9puzzle/status/1018120012966621184

![The puzzle](https://pbs.twimg.com/media/DiEWj8gXkAIXFeE.jpg)

### My solution

In [4]:
from itertools import product, permutations

In [5]:
# Find the correct labels for all the sums we might possibly encounter
sum_dict = {}
max_sum = 9+8+7+6

# Evens and odds
for k in range(max_sum//2+1): sum_dict[2*k] = 'E'
for k in range(max_sum//2+1): sum_dict[2*k+1] = 'O'

# Primes (bleh!)
primes = list(range(2,max_sum+1))
for j in range(2,int(max_sum**.5)+1):
    for i in range(2,max_sum//j+1):
        try:
            primes.remove(i*j)
        except:
            pass
for k in primes: sum_dict[k] = 'P'
    
# Square, cube, triangular numbers
for k in range(int(max_sum**.5)+1): sum_dict[k**2] = 'S'
for k in range(int(max_sum**.3334)+1): sum_dict[k**3] = 'C'
for k in range(1,int((max_sum*2+1)**.5)+1): sum_dict[k*(k+1)//2] = 'T'
    

In [6]:
def checkgrid(grid):
    '''Determine whether a given grid obeys the rules.'''
    for i,j in product(range(7), range(7)):
        if (type(grid[i][j]) == str 
            and grid[i][j] != sum_dict[
                grid[i+1][j] + grid[i-1][j] 
                + grid[i][j+1] + grid[i][j-1]
            ]):
            return False
    return True

def printgrid(grid):
    '''Neatly print out a grid.'''
    print(('\n' + '-' * (4*len(grid[0]) - 3) + '\n')
          .join([' | '.join(str(x) for x in g) for g in grid]))  
    

In [7]:
# Set up the grid for the puzzle
grid = [[0] * 7 for _ in range(7)]

# Label the dark squares in the puzzle grid
grid[1][4] = 'T'
grid[2][1] = 'C'
grid[2][3] = 'T'
grid[3][2] = 'E'
grid[3][4] = 'E'
grid[4][3] = 'E'
grid[4][5] = 'E'
grid[5][2] = 'P'
grid[3][3] = 7

# Prepare to handle the light squares in the puzzle grid
lights = [(i,j) for i,j in product(range(1,6), range(1,6)) 
          if (i+j) % 2 == 0
          and (i,j) not in [(1,5),(5,1),(1,1),(5,5),(3,3)]]

# Try each possible way to fill the light squares, and show the solution(s)
for digits in permutations([1,2,3,4,5,6,8,9]):
    for (i,j),d in zip(lights, digits):
        grid[i][j] = d
    if checkgrid(grid):
        printgrid(grid)
        print('\n\n')
        

0 | 0 | 0 | 0 | 0 | 0 | 0
-------------------------
0 | 0 | 0 | 2 | T | 0 | 0
-------------------------
0 | C | 5 | T | 1 | 0 | 0
-------------------------
0 | 3 | E | 7 | E | 8 | 0
-------------------------
0 | 0 | 9 | E | 6 | E | 0
-------------------------
0 | 0 | P | 4 | 0 | 0 | 0
-------------------------
0 | 0 | 0 | 0 | 0 | 0 | 0





## "1 to 9" Puzzle #180907
Source: https://twitter.com/1to9puzzle/status/1037836364262395904

![The puzzle](https://pbs.twimg.com/media/DmciqVZW4AAruCj.jpg)

### My solution
I'll reuse some of the tools from previous problems:  
`sum_dict`, `cols()`, `printgrid()`, `itertools`

In [8]:
def checkgrid(grid, row_reqs, col_reqs, diag_reqs):
    '''Determine whether a given grid fits the requirements.'''
    
    # Check the rows and columns
    for i, row in enumerate(grid):
        if sum_dict[sum(row)] != row_reqs[i]:
            return False
    for i, col in enumerate(cols(grid)):
        if sum_dict[sum(col)] != col_reqs[i]:
            return False

    # Check the primary diagonal
    n = len(grid)
    if sum_dict[sum(grid[i][i] for i in range(n))] != diag_reqs[0]:
        return False

    # Check the secondary diagonal
    if sum_dict[sum(grid[i][n-i-1] for i in range(n))] != diag_reqs[1]:
        return False
    
    return True    

In [9]:
# Set up the grid for the puzzle
grid = [[0] * 3 for _ in range(3)]
grid[1][1] = 4

# Prepare to handle the blank squares in the puzzle grid
blanks = [(i,j) for i,j in product(range(3), range(3)) if i*j != 1]

# Try each possible way to fill the blanks, and show the solution(s)
for digits in permutations([1,2,3,5,6,7,8,9]):
    for (i,j),d in zip(blanks, digits):
        grid[i][j] = d
    if checkgrid(
        grid,
        row_reqs  = ('S','E','P'),
        col_reqs  = ('T','S','E'),
        diag_reqs = ('E','T')
    ):
        printgrid(grid)
        print('\n\n')
        

2 | 9 | 5
---------
7 | 4 | 1
---------
6 | 3 | 8



8 | 3 | 5
---------
1 | 4 | 7
---------
6 | 9 | 2





## "1 to 9" Puzzle #1836
Source: https://twitter.com/1to9puzzle/status/1038053770561826816

![The puzzle](https://pbs.twimg.com/media/DmfngHlXoAADa78.jpg)

### My solution

Again I'll reuse a few tools from earlier:   
`checkgrid()`, `printgrid()`, `itertools`  
In fact, it turns out that this problem requires almost nothing more than to redefine the `sum_dict`!

In [10]:
import inflect

In [11]:
# Find the correct labels for all the sums we might possibly encounter
# Each sum is represented by the first letter of the sum when 
# written out in English (e.g. represent 6 + 1 + 7 with "F")
sum_dict = {}
num2word = inflect.engine().number_to_words
for k in range(9+8+7+1): 
    sum_dict[k] = num2word(k)[0].upper()

In [12]:
# Set up the grid for the puzzle
grid = [[0] * 3 for _ in range(3)]
grid[1][1] = 2

# Prepare to handle the blank squares in the puzzle grid
blanks = [(i,j) for i,j in product(range(3), range(3)) if i*j != 1]

# Try each possible way to fill the blanks, and show any solution(s)
for digits in permutations([1,3,4,5,6,7,8,9]):
    for (i,j),d in zip(blanks, digits):
        grid[i][j] = d
    if checkgrid(
        grid,
        row_reqs  = ('N', 'E', 'F'),
        col_reqs  = ('S', 'S', 'T'),
        diag_reqs = ('S', 'S'),
    ):
        printgrid(grid)
        print('\n\n')
        

3 | 9 | 7
---------
5 | 2 | 4
---------
8 | 6 | 1



7 | 3 | 9
---------
4 | 2 | 5
---------
6 | 1 | 8





## "Skyscrapers"
Source: https://twitter.com/solvemymaths/status/1038308216264896518

![The puzzle part 1](https://pbs.twimg.com/media/DmjP0LaXgAAPh0X.jpg)
![The puzzle part 2](https://pbs.twimg.com/media/DmjP0ahX0AEUN2K.jpg)


### My solution

We'll reuse some code from before verbatim, but let's pretend we're starting from scratch.

In [13]:
from copy import deepcopy

In [14]:
def count_maxes(l):
    '''Count the number of times a list of positive numbers reaches a new maximum.'''
    mx = l[0]
    cnt = 1
    for i in l[1:]:
        if i and i > mx:
            cnt += 1
            mx = i
    return cnt

def checkrow(row, req):
    '''Make sure a given row (or column) isn't breaking its two given requirements.'''

    # The row may contain up to one of each number
    n = len(row)
    if max(row.count(d+1) for d in range(n)) > 1:
        return False
    
    # If the row has any numbers, check that they do not exceed the first requirement
    if req[0] and any(row) and count_maxes(row) > req[0]:
        return False
    
    # If the row is full, check that both requirements are met exactly
    if req[0] and all(row) and count_maxes(row) != req[0]:
        return False
    if req[1] and all(row) and count_maxes(row[::-1]) != req[1]:
        return False
    
    return True

def cols(grid):
    '''Get all the columns of a grid (essentially, transpose the grid).'''
    return [[e[i] for e in grid] for i in range(len(grid[0]))]

def col(grid, k):
    '''Get a single column of a grid.'''
    return [e[k] for e in grid]

def checkgrid(grid, row_reqs, col_reqs):
    '''Check that no rules are violated in an entire grid.'''
    if not all(checkrow(g, r) for g,r in zip(grid, row_reqs)):
        return False
    if not all(checkrow(g, r) for g,r in zip(cols(grid), col_reqs)):
        return False
    return True

def printgrid(grid):
    '''Neatly print out a grid.'''
    print(('\n' + '-' * (4*len(grid[0]) - 3) + '\n')
          .join([' | '.join(str(i) for i in g) for g in grid]))  

    
def solve_it(row_reqs, col_reqs, fill_coords = [], fill_vals = []):
    '''For a skyscraper problem specified by certain row and 
    column requirements, find and display all solutions.'''

    # Size of the problem
    n = len(row_reqs)

    # In each spot in a grid, numbers will be tested in order
    cycle = {None : 1}
    for i in range(1,n): cycle[i] = i+1

    solutions = []

    # The initial grid with given values filled in
    grid = [[None] * n for _ in range(n)]
    for (i,j), v in zip(fill_coords, fill_vals):
        grid[i][j] = v
    
    # Starting position in the grid (upper left)
    currx, curry = 0, 0

    # Keep working until all possibilites are exhausted
    # (so if there are multiple solutions, we can find all of them)
    while curry >= 0:
                
        # At the current position in the grid, try the next letter
        try:
            grid[curry][currx] = cycle[grid[curry][currx]]

        # If there are no more letters to try, back up to the previous position
        # (and keep backing up if a filled spot is reached)
        except KeyError: 
            grid[curry][currx] = None
            retreat = True
            while retreat:
                currx -= 1
                if currx < 0:
                    currx  = n-1
                    curry -= 1
                retreat = ((curry, currx) in fill_coords)
            continue

        # If we have found a solution, save it
        if (currx, curry) == (n-1,n-1) and checkgrid(grid, row_reqs, col_reqs):
            solutions.append(deepcopy(grid))
            
        # Otherwise, if we're not currently breaking any rules, advance to the next position
        # (and keep advancing if a filled spot is reached)        
        elif (checkrow(grid[curry], row_reqs[curry]) and 
              checkrow(col(grid, currx), col_reqs[currx])):
            advance = True
            while advance:
                currx += 1
                if currx > n-1:
                    currx  = 0
                    curry += 1
                advance = ((curry, currx) in fill_coords)
    
    return solutions


#### First puzzle

In [15]:
soln = solve_it(
    row_reqs = [(2,3),(2,2),(2,1),(1,4)],
    col_reqs = [(4,1),(1,2),(2,2),(3,2)]
)

for s in soln: 
    printgrid(s)
    print('\n\n')


1 | 4 | 3 | 2
-------------
2 | 1 | 4 | 3
-------------
3 | 2 | 1 | 4
-------------
4 | 3 | 2 | 1





#### Second puzzle

In [16]:
soln = solve_it(
    row_reqs = [(3,None),(None,None),(2,None),(None,None)],
    col_reqs = [(3,None),(None,None),(2,None),(None,None)],
    fill_coords = [(1,2)], fill_vals = [1]
)

for s in soln: 
    printgrid(s)
    print('\n\n')


1 | 3 | 2 | 4
-------------
3 | 4 | 1 | 2
-------------
2 | 1 | 4 | 3
-------------
4 | 2 | 3 | 1



